# ESTA ES LA CLASE 

## Redes Recurrentes - Sentiment Analysis

Sec2Vec o Sec2Sec

Sirven para información secuencial.

Tiene "memoria" de lo que vio antes: Con una conexión recurrente.

En estos casos la entrada es $X_t$ y la salida es $X_{t+1}$


SRNN (vanilla) -> Elman

### Doble entrada, una que es la $X_t$ y otra que es la anterior capa $h_{t-1}$

### La salida es $X_{t+1}$ y $h_{t+1}$

#### El dataset es una base de datos de reseñas de IMDB con tags positivos o negativos

Hay que armar un vocabulario de palabras y asignarle un indice a cada palabra

Algunos análisis suelen tener en cuenta la frecuencia de palabras

Se suelen cortar las palabras según su frecuencia, si cortamos articulos a partir de cierta frecuancia

O Se puede cortar la cola de frecuencias, las que aparecen 1 o 2 veces no son significativas

### Armamos un one_hot de N-1 0 y 1 en la posición $w_i$ donde $w_i$ es la posición correspondiente a la palabra

### $X_t$ es el one_hot_vector. Con estas $X_t$ se genera el entendimiento del lenguaje.

### Esta red genera una salida $h_f$ que se va a usar en un clasificador para ver si es positiva o negativa

![alt text](RNN.png "Title")

### El ciclo es: 


### - Meto $X_t$ sumo el $h_{t-1}$ genero un $Y$ 
### - Lo retropropago con el $X_{t+1}$
### - Luego meto el $x_{t+1}$ y el $h_{t}$ para generar el  $Y_{t+1}$ que se compara con el $X_{t+2}$ ETC...
### - Una vez que pase toda la secuencia, uso $h_f$ para clasificar si es 1 o 0 y retropropago para TODOS.


### Vemos todo como un clasificador y usamos CrossEntropy

### Entrenamos los pesos de $X$ a $h$, $h_{t-1}$ a $h_t$, de $h$ a $Y$ y de $h_f$ a output

### Este anda mejor con SGD pero los que son mejores funcionan mejor con Minibatch

## LSA, Latent Semantic Analysis



# Word Embedding

## Palabras semanticamente parecidas tienen embedding parecidos

## Word2vec :  A partir de una palabra se fija el contexto.
## KING - MAN + WOMAN = QUEEN

## Los embeddings de torch toma hasta -2 palabras como contexto.

### INPUT - EMBEDDING - RNN 


#### Para RNN uso dropout para que cada neurona tenga features especificas y no que cada una afecte un poquito, para eso va el model.eval

# LSTM devuelve (xo , h , c)


## Proba entrenar mientras probas

# Red hecha masomenos a mano

In [2]:
from reviewer import *
from recurrent_a import *
import matplotlib.pyplot as plt

f = !ls ./data/*labelled.txt

reviews = load(f[0])
word_list , word_to_ix = vocabulary(reviews)
seq = sequence(reviews, word_to_ix)
P = len(reviews)
N = len(word_to_ix)
T = 2

#RNN_train(N,T,P, seq)
    
    


# Como hacer tu propio dataset

Todo viene del archivo recurrent_b.py

In [1]:
reviews = load("./data/imdb_labelled.txt")

wl, wd = vocabulary(reviews)

seqs = sequence(reviews, wd)


NameError: name 'load' is not defined

In [7]:

reviews = load( "./data/imdb_labelled.txt")
wl, wd = vocabulary( reviews)
seqs = sequence( reviews, wd)

P = len(reviews)
N = len(wl)
T = 10
B = 50
L = 75      # Review Max. Len.

#Necesita que la matriz P x L sea long int
pads = torch.LongTensor( P, L)
for i in range(P):
    # Setea todo en 0
    seq = seqs[i][0]
    M = len(seq)
    pads[i] = 0
    if M<L:
        pads[i,-M:] = torch.tensor(seq)
    else:
        pads[i] = torch.tensor(seq[:L])

        
# Guarda los labels
labels = torch.tensor([ label for words,label in reviews], dtype=torch.float)

# DEJO EL 25% PARA EL TEST
trn_inputs =   pads[:-N//4]
trn_target = labels[:-N//4]

tst_inputs =   pads[-N//4:]
tst_target = labels[-N//4:]

## Esta linea la dejo asi, funciones virtuales...
from torch.utils.data import TensorDataset, DataLoader

trn_data = TensorDataset( trn_inputs, trn_target)
tst_data = TensorDataset( tst_inputs, tst_target)

trn_load = DataLoader( trn_data, shuffle=True, batch_size=B)
tst_load = DataLoader( tst_data, shuffle=True, batch_size=B)




class Sentiment( torch.nn.Module):
    def __init__( _, vocab, embed, context, output=1):
        super().__init__()
        _.isize = vocab
        _.esize = embed
        _.hsize = context
        _.osize = output
        
        _.embedding =  torch.nn.Embedding(_.isize, _.esize) # 
        
        # num_layers, dropout apaga neuronas al azar, bidirectional = True entrena otra red al mismo tiempo con las flechas al reves
        # esize es el tamaño del embedding y hsize es la salida
        _.rnn = torch.nn.GRU(_.esize, _.hsize, batch_first = True) # Cantidad de capas que quiero adentro
        
        
        _.l = torch.nn.Linear(_.hsize,_.osize)

    def forward( _, xi):
        
        print(xi)
        
        xe = _.embedding(xi)
        
        # xo son todos los h por los que paso la red y h es el ultimo h de salida
        xo, h = _.rnn(xe)
        
        y = torch.tanh(_.l(h))
        
        # squeeze saca todas las dimensiones extras y lo deja como vector
        return y.squeeze()


model = Sentiment( N, 200, 200, 1)
optim = torch.optim.Adam( model.parameters())
lostf = torch.nn.MSELoss()


model.train()
err = []
for t in range(T):
    E = 0.
    
    for words, label in trn_load:

        optim.zero_grad() 
        
        y = model(words)
        loss = lostf(y,label)

        loss.backward()
        
        err.append(loss.item())


        optim.step()
        
        E += loss.item()
        
    print( t, E)
plt.plot(err)
plt.show()

model.eval()
with torch.no_grad():
    r, t = 0, 0
    for words, label in tst_load:
        senti = model(words)
        r += ( (senti>=0.5)==(label==1) ).sum().item()
        t += len(label)
print( "Accuracy:", 100*r/t)


tensor([[   0,    0,    0,  ...,  161, 1851,  621],
        [   0,    0,    0,  ..., 1441, 1613,  621],
        [   0,    0,    0,  ..., 1257, 2841,  621],
        ...,
        [   0,    0,    0,  ..., 1274, 2760,  621],
        [   0,    0,    0,  ..., 1057, 2180,  621],
        [   0,    0,    0,  ..., 2143, 2086,  621]])
tensor([[   0,    0,    0,  ...,  659, 1668,  621],
        [   0,    0,    0,  ...,  891,  190,  621],
        [   0,    0,    0,  ..., 2521, 2519,  621],
        ...,
        [   0,    0,    0,  ...,  968, 2941,  621],
        [   0,    0,    0,  ...,  971,  130,  621],
        [   0,    0,    0,  ..., 2426,  102,  621]])
tensor([[   0,    0,    0,  ..., 1717,  397,  621],
        [   0,    0,    0,  ...,  593,  645,  621],
        [   0,    0,    0,  ..., 2430, 1057,  621],
        ...,
        [   0,    0,    0,  ...,  940, 2987,  621],
        [   0,    0,    0,  ..., 1195, 1948,  621],
        [   0,    0,    0,  ..., 2758,  241,  621]])
tensor([[   0,    0,  

KeyboardInterrupt: 